In [1]:
import numpy as np
from nim_env import NimEnv, OptimalPlayer

# Nim environment

Our 2nd game is the famous game of Nim. You can read about the game and its rules here: https://en.wikipedia.org/wiki/Nim

**Important note:** We consider the normal (i.e. not misere) game: the player taking the last object *wins*.

We implemented the game as an environment in the style of games in the [Python GYM library](https://gym.openai.com/). The commented source code is available in the file "nim_env.py". Here, we give a brief introduction to the environment and how it can be used.

### Initialization and attributes

Given a random seed, you can initialize the environment / game in a random state (3 heaps with between 1 to 7 sticks in each heap) as following:

In [2]:
env = NimEnv(seed = 3)

Which then has the following attributes with the corresponding initial values:

In [3]:
env.__dict__

{'n_heap': 3,
 'n_agents': 2,
 'current_player': 0,
 'winner': None,
 'end': False,
 'num_step': 0,
 'heaps': [2, 5, 6],
 'heap_avail': [True, True, True],
 'heap_keys': ['1', '2', '3']}

The game is played by two players: player 0 and player 1. The attribute 'current_player' shows whose turn it is. We assume that player 0 always plays first.

The attribute 'heaps' is a numpy array of size 3 and presents the board in the real game and the state $s_t$ in the reinfocement learning language. Each number shows available number of sticks in each heap. The attribute 'heap_avail' shows which heaps can be used. 
        
The attribute 'end' shows if the game is over or not, and the attribute 'winner' shows the winner of the game.

You can use function 'render' to visualize the current position of the board:

In [4]:
env.render()

───────────────────────────────────
Heap 1: ||              	 (2)
───────────────────────────────────
Heap 2: |||||           	 (5)
───────────────────────────────────
Heap 3: ||||||          	 (6)
───────────────────────────────────


### Taking actions

The game environment will recieve action from two players in turn and update the heaps. At each time, one player can take the action $a_t$, where $a_t$=action can is a vector of 2 integer: action[0] $\in \{ 1,2,3 \}$ is the number of heap and action[1] > 0 the number of sticks to be taken from that heap.

Function 'step' is used to recieve the action of the player, update the grid:

In [5]:
env.step([1,2])

([0, 5, 6], False, None)

In [6]:
env.render()

───────────────────────────────────
Heap 1:                 	 (0)
───────────────────────────────────
Heap 2: |||||           	 (5)
───────────────────────────────────
Heap 3: ||||||          	 (6)
───────────────────────────────────


In [7]:
env.__dict__

{'n_heap': 3,
 'n_agents': 2,
 'current_player': 1,
 'winner': None,
 'end': False,
 'num_step': 1,
 'heaps': [0, 5, 6],
 'heap_avail': [False, True, True],
 'heap_keys': ['1', '2', '3']}

In [8]:
env.step([2,3])

([0, 2, 6], False, None)

In [9]:
env.render()

───────────────────────────────────
Heap 1:                 	 (0)
───────────────────────────────────
Heap 2: ||              	 (2)
───────────────────────────────────
Heap 3: ||||||          	 (6)
───────────────────────────────────


In [10]:
env.__dict__

{'n_heap': 3,
 'n_agents': 2,
 'current_player': 0,
 'winner': None,
 'end': False,
 'num_step': 2,
 'heaps': [0, 2, 6],
 'heap_avail': [False, True, True],
 'heap_keys': ['1', '2', '3']}

But not all actions are available at each time: One cannot take sticks from an unavailable action is taken. There is an error if an unavailable action is taken:

In [11]:
env.step([2,3])

AssertionError: You cannot take more objects than there are in the heap

Not taking any stick is also unavailable:

In [12]:
env.step([2,0])

AssertionError: You must take at least 1 object from the heap

### Reward

Reward is always 0 until the end of the game. When the game is over, the reward is 1 if you win the game and -1 if you lose. Function 'observe' can be used after each step to recieve the new state $s_t$, whether the game is over, and the winner, and function 'reward' to get the reward value $r_t$:

In [13]:
env.observe()

([0, 2, 6], False, None)

In [14]:
env.reward(player=0)

0

In [15]:
env.reward(player=1)

0

An example of finishing the game:

In [16]:
print("Player = " + str(env.current_player))
env.step([2,2])
env.render()
print("Player = " + str(env.current_player))
env.step([3,6])
env.render()

Player = 0
───────────────────────────────────
Heap 1:                 	 (0)
───────────────────────────────────
Heap 2:                 	 (0)
───────────────────────────────────
Heap 3: ||||||          	 (6)
───────────────────────────────────
Player = 1
───────────────────────────────────
Heap 1:                 	 (0)
───────────────────────────────────
Heap 2:                 	 (0)
───────────────────────────────────
Heap 3:                 	 (0)
───────────────────────────────────


In [17]:
env.observe()

([0, 0, 0], True, 1)

In [18]:
env.reward(player=0)

-1

In [19]:
env.reward(player=1)

1

# Optimal policy for the Nim environment

Fortunately, we know the exact optimal policy for Nim. We have implemented and $\epsilon$-greedy version of optimal polciy which you can use for the project.

In [5]:
env.reset(seed=6);

In [6]:
env.render()

───────────────────────────────────
Heap 1: |||||||         	 (7)
───────────────────────────────────
Heap 2: |||||           	 (5)
───────────────────────────────────
Heap 3: |               	 (1)
───────────────────────────────────


In [7]:
env.__dict__

{'n_heap': 3,
 'n_agents': 2,
 'current_player': 0,
 'winner': None,
 'end': False,
 'num_step': 0,
 'heaps': [7, 5, 1],
 'heap_avail': [True, True, True],
 'heap_keys': ['1', '2', '3']}

In [8]:
opt_player = OptimalPlayer(epsilon = 0., player = 0)

In [9]:
opt_player.act(env.heaps)

[1, 3]

In [10]:
opt_player.player

0

### An example of optimal player playing against random player

In [11]:
Turns = np.array([0,1])
for i in range(5):
    env.reset()
    heaps, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt = OptimalPlayer(epsilon=0., player=Turns[0])
    player_rnd = OptimalPlayer(epsilon=1., player=Turns[1])
    while not env.end:
        if env.current_player == player_opt.player:
            move = player_opt.act(heaps)
        else:
            move = player_rnd.act(heaps)

        heaps, end, winner = env.step(move)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player = ' +  str(Turns[0]))
            print('Random player = ' +  str(Turns[1]))
            env.reset()
            break


-------------------------------------------
Game end, winner is player 0
Optimal player = 1
Random player = 0
-------------------------------------------
Game end, winner is player 0
Optimal player = 0
Random player = 1
-------------------------------------------
Game end, winner is player 0
Optimal player = 0
Random player = 1
-------------------------------------------
Game end, winner is player 0
Optimal player = 0
Random player = 1
-------------------------------------------
Game end, winner is player 0
Optimal player = 0
Random player = 1


### An example of optimal player playing against optimal player

In [12]:
Turns = np.array([0,1])
for i in range(5):
    env.reset()
    heaps, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt_1 = OptimalPlayer(epsilon=0., player=Turns[0])
    player_opt_2 = OptimalPlayer(epsilon=0., player=Turns[1])
    while not env.end:
        if env.current_player == player_opt_1.player:
            move = player_opt_1.act(heaps)
        else:
            move = player_opt_2.act(heaps)

        heaps, end, winner = env.step(move)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player 1 = ' +  str(Turns[0]))
            print('Optimal player 2 = ' +  str(Turns[1]))
            env.reset()
            break


-------------------------------------------
Game end, winner is player 0
Optimal player 1 = 1
Optimal player 2 = 0
-------------------------------------------
Game end, winner is player 0
Optimal player 1 = 1
Optimal player 2 = 0
-------------------------------------------
Game end, winner is player 1
Optimal player 1 = 0
Optimal player 2 = 1
-------------------------------------------
Game end, winner is player 1
Optimal player 1 = 0
Optimal player 2 = 1
-------------------------------------------
Game end, winner is player 0
Optimal player 1 = 1
Optimal player 2 = 0
